<a href="https://colab.research.google.com/github/Nablasquarednull/BCH_proyect/blob/main/BCH_stable_ver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
from operator import indexOf
import sympy
from sympy import symbols
from sympy.plotting import plot
sympy.init_printing()
from sympy import*
from sympy.physics.quantum import *
from sympy.physics.quantum import Operator, Commutator, AntiCommutator
from sympy.physics.quantum import Dagger, Commutator
from sympy.physics.quantum.boson import BosonOp
from math import ceil
from sympy.printing.latex import LatexPrinter, print_latex
#-----------------------------
#-----------------------------

def collect_coeff(expresion,A): #factoriza el operador A de una expresión "expresion"
  vt = 0
  for k in range(len(expand(expresion).args)):
    if (expand(expresion).args[k]).coeff(A)!=0:
      vt+= (expand(expresion).args[k].coeff(A))
  return (vt)*A
#--------------------------------------
def collect_coeff2(expresion,A): #factoriza el operador A de una expresión "expresion" NOTA AL DR. RAMOS: MODIFIQUÉ LA EXPRESIÓN FINAL: AHORA REGRESA (vt), antes regresaba (vt)*A
  vt = 0
  for k in range(len(expand(expresion).args)):
    if (expand(expresion).args[k]).coeff(A)!=0:
      vt+= (expand(expresion).args[k].coeff(A))
  return (vt)
#-----------------------------
def conmutador(A,B):
  return Commutator(A,B).expand(commutator=True).expand(commutator=True).subs(commutator_subs)
#-----------------------------
def com_order(A,B,n): #regresa el conmutador anidado [A,[A,..[A,B]] n veces.
    if n<=1:
        return conmutador(A,B)

    else:
        return com_order(A,conmutador(A,B),n-1)
#-----------------------------
class MyLatexPrinter(LatexPrinter):
    """Print derivative of a function of symbols in a shorter form.
    """
    def _print_Derivative(self, expr):
        function, *vars = expr.args
        if not isinstance(type(function), UndefinedFunction) or \
           not all(isinstance(i, Symbol) for i in vars):
            return super()._print_Derivative(expr)

        # If you want the printer to work correctly for nested
        # expressions then use self._print() instead of str() or latex().
        # See the example of nested modulo below in the custom printing
        # method section.
        return "{}_{{{}}}".format(
            self.print(Symbol(function.func.name_)),
                        ''.join(self._print(i) for i in vars))


def Latex(expr):
    """ Most of the printers define their own wrappers for print().
    These wrappers usually take printer settings. Our printer does not have
    any settings.
    """
    print(MyLatexPrinter().doprint(expr))
#------------------------------
def BCH(A, B, alpha, n): # Baker_Campbell_Hausdorff formula
    a=0
    for i in range(1,n+1):
        a += (alpha**i/factorial(i))*com_order(A,B,i)
    return (B + a)
#-------------------------------
def collect_all(expression,operator_list): # itera "len(operator list)" veces la función "collect_coef" y suma todas sus iteraciones.
  a=0
  for i in range(len(operator_list)):
    a += collect_coeff(expression,operator_list[i])
  return a
#-------------------------------
def collect_and_save_all(expression,operator_list):
  coeff_list=[] #lista donde vamos a meter los coeficientes de cada operador
  for z in range(len(operator_list)): #iterador para cada operador
    a = collect_coeff2(expression,operator_list[z])
    coeff_list.append(a)
  return coeff_list
#-------------------------------
def compare_all(coeff_list,operator_list,n): #compara los coeficientes y los reemplaza.
  list_1= coeff_list.copy()
  functions = [cos, sin , exp] #lista de funciones para comparar
  for z in range(len(operator_list)):
    for i in functions: #iterador para cada funcion
      if coeff_list[z] == series(i(omega*t),omega*t,0,n).removeO(): #funciones en 'functions' positivas
        list_1[z] = i(omega*t)
        break
      elif coeff_list[z] == series(-i(omega*t),omega*t,0,n).removeO():# funciones en 'functions' negativas
        list_1[z] = -i(omega*t)
        break
      elif coeff_list[z] == fraction(series(i(omega*t)/omega,omega,0,n).removeO())[0]: #funciones en 'functions' cardinales positivas
        list_1[z] = i(omega*t)/omega
        break
      elif coeff_list[z] == fraction(series(-i(omega*t)/omega,omega,0,n).removeO())[0]:#funciones en 'functions' cardinales negativas
        list_1[z] = -i(omega*t)/omega
        break
      elif coeff_list[z] == series(i(omega*t)*omega,omega,0,n).removeO(): #funciones en 'functions' cardinales positivas
        list_1[z] = i(omega*t)*omega
        break
      elif coeff_list[z] == series(-i(omega*t)*omega,omega,0,n).removeO():#funciones en 'functions' cardinales negativas
        list_1[z] = -i(omega*t)*omega
        break
      elif coeff_list[z] == series(i(omega*t)*omega,omega,0,n+1).removeO(): #funciones en 'functions' cardinales positivas
        list_1[z] = i(omega*t)*omega
        break
      elif coeff_list[z] == series(-i(omega*t)*omega,omega,0,n+1).removeO():#funciones en 'functions' cardinales negativas
        list_1[z] = -i(omega*t)*omega
        break
      elif coeff_list[z] == series(i(omega*t)*omega,omega,0,n+2).removeO(): #funciones en 'functions' cardinales positivas
        list_1[z] = i(omega*t)*omega
        break
      elif coeff_list[z] == series(-i(omega*t)*omega,omega,0,n+2).removeO():#funciones en 'functions' cardinales negativas
        list_1[z] = -i(omega*t)*omega
        break
  return list_1

#-------------------------------
def sum_collect(coef_list,operator_list):
  a=0
  for i in range(len(operator_list)):
    a += coef_list[i]*(operator_list[i])
  return a
#-------------------------------
def extract_operator_and_coeff(expression):
    """
    Extracts the operator and coefficient from an expression of the form exp(coeff * operator).

    Args:
    expression: The expression to extract from.

    Returns:
    A tuple containing the operator and the coefficient.
    """
    # Get the argument of the exponential function
    arg = expression.args[0]

    # If the argument is a multiplication, extract the operator and coefficient
    if arg.is_Mul:
        operator = arg.args[-1]  # Assume the operator is the last argument
        coeff = arg / operator   # Get the coefficient
    else:
        operator = arg
        coeff = 1

    return operator, coeff
    #-------------------------------
def time_dep_exp(BCH,transformation):
  return BCH -I*Dagger(transformation)*transformation.diff(t)

#----------------------------------------------------------
#Symbols
omega = symbols(r'\omega', real = True)
alpha = symbols(r'\alpha')
t     = symbols(r't', real = True)
#Operators
x     = Operator(r'\hat{x}')
p     = Operator(r'\hat{p}')
H     = (p**2+omega**2*x**2)/(2)

l =[x,p]

H_2 = sigmaz + sigmap + sigmam
l_2 = [sigmaz,sigmap,sigmam]
sigmap = Operator(r'\hat{\sigma}_+')
sigmam = Operator(r'\hat{\sigma}_-')
sigmaz = Operator(r'\hat{\sigma}_z')

a_dag = Operator(r'\hat{a}^{\dagger}')
a = Operator(r'\hat{a}')
H_3 = omega*(a_dag*a+1/2)
l_3 = [a,a_dag]
#Functions

rho = Function(r'\rho')(t)
prueba = exp((I*rho.diff(t)/(2*rho))*x**2)




#---------------------------------------

#---------------------------------------
commutator_subs = {
    Commutator(x, p): I,
    Commutator(p, x): -I,
    Commutator(p, p): 0,
    Commutator(x, x): 0,

    Commutator(a, a_dag): 1,
    Commutator(a_dag, a): -1,
    Commutator(a_dag, a_dag): 0,
    Commutator(a, a): 0,

    Commutator(sigmap, sigmam): sigmaz,
    Commutator(sigmam, sigmap): -sigmaz,

    Commutator(sigmaz, sigmap): 2 * sigmap,
    Commutator(sigmap, sigmaz): -2 * sigmap,
    Commutator(sigmam, sigmaz): 2 * sigmam,
    Commutator(sigmaz, sigmam): -2 * sigmam,

    Commutator(sigmap, sigmap): 0,
    Commutator(sigmam, sigmam): 0,
    Commutator(sigmaz, sigmaz): 0,
}


In [50]:
expand(time_dep_exp(BCH(H,x,alpha,9),prueba))

                                                                                                   ↪
                                                                                                   ↪
                                                                                                   ↪
                                                                                                   ↪
                                                                                                   ↪
                                                                                                   ↪
                                                                                                   ↪
                                                                                                   ↪
          9       4                 8       4                   7       3                 6        ↪
  ⅈ⋅\alpha ⋅\omega ⋅\hat{p}   \alpha ⋅\omega ⋅\hat{x}   ⅈ⋅\alpha ⋅\omega ⋅\hat{p}   \alpha 

In [24]:
BCH_X = BCH(H,x,alpha,9).subs({alpha:I*t})
BCH_X

      8  9                 8  8                 6  7                 6  6                 4  5     ↪
\omega ⋅t ⋅\hat{p}   \omega ⋅t ⋅\hat{x}   \omega ⋅t ⋅\hat{p}   \omega ⋅t ⋅\hat{x}   \omega ⋅t ⋅\ha ↪
────────────────── + ────────────────── - ────────────────── - ────────────────── + ────────────── ↪
      362880               40320                 5040                 720                  120     ↪

↪              4  4                 2  3                 2  2                              
↪ t{p}   \omega ⋅t ⋅\hat{x}   \omega ⋅t ⋅\hat{p}   \omega ⋅t ⋅\hat{x}                      
↪ ──── + ────────────────── - ────────────────── - ────────────────── + t⋅\hat{p} + \hat{x}
↪                24                   6                    2                               

In [25]:
BCH_X_1 = collect_all(BCH_X,l)
BCH_X_1

⎛      8  8         6  6         4  4         2  2    ⎞           ⎛      8  9         6  7         ↪
⎜\omega ⋅t    \omega ⋅t    \omega ⋅t    \omega ⋅t     ⎟           ⎜\omega ⋅t    \omega ⋅t    \omeg ↪
⎜────────── - ────────── + ────────── - ────────── + 1⎟⋅\hat{x} + ⎜────────── - ────────── + ───── ↪
⎝  40320         720           24           2         ⎠           ⎝  362880        5040         12 ↪

↪  4  5         2  3    ⎞        
↪ a ⋅t    \omega ⋅t     ⎟        
↪ ───── - ────────── + t⎟⋅\hat{p}
↪ 0           6         ⎠        

In [27]:
BCH_X_2 = collect_and_save_all(BCH_X,l)
BCH_X_2

⎡      8  8         6  6         4  4         2  2            8  9         6  7         4  5       ↪
⎢\omega ⋅t    \omega ⋅t    \omega ⋅t    \omega ⋅t       \omega ⋅t    \omega ⋅t    \omega ⋅t    \om ↪
⎢────────── - ────────── + ────────── - ────────── + 1, ────────── - ────────── + ────────── - ─── ↪
⎣  40320         720           24           2             362880        5040         120           ↪

↪    2  3    ⎤
↪ ega ⋅t     ⎥
↪ ─────── + t⎥
↪  6         ⎦

In [28]:
BCH_X_3 = compare_all(BCH_X_2,l,9)
BCH_X_3

⎡               sin(\omega⋅t)⎤
⎢cos(\omega⋅t), ─────────────⎥
⎣                  \omega    ⎦

In [9]:
BCH_X_4 = sum_collect(BCH_X_3,l)
BCH_X_4

                        sin(\omega⋅t)⋅\hat{p}
cos(\omega⋅t)⋅\hat{x} + ─────────────────────
                               \omega        

In [10]:
BCH_P = BCH(H,p,alpha,3).subs({alpha:I*t})
BCH_P

      4  3                 2  2                                      
\omega ⋅t ⋅\hat{x}   \omega ⋅t ⋅\hat{p}         2                    
────────────────── - ────────────────── - \omega ⋅t⋅\hat{x} + \hat{p}
        6                    2                                       

In [ ]:
BCH_P_1 = collect_all(BCH_P,l)
BCH_P_1

⎛        2  2    ⎞           ⎛      4  3            ⎞        
⎜  \omega ⋅t     ⎟           ⎜\omega ⋅t          2  ⎟        
⎜- ────────── + 1⎟⋅\hat{p} + ⎜────────── - \omega ⋅t⎟⋅\hat{x}
⎝      2         ⎠           ⎝    6                 ⎠        

In [ ]:
BCH_P_2 = collect_and_save_all(BCH_P,l)
BCH_P_2

⎡      4  3                      2  2    ⎤
⎢\omega ⋅t          2      \omega ⋅t     ⎥
⎢────────── - \omega ⋅t, - ────────── + 1⎥
⎣    6                         2         ⎦

In [ ]:
BCH_P_3 = compare_all(BCH_P_2,l,3)
BCH_P_3

[-\omega⋅sin(\omega⋅t), cos(\omega⋅t)]

In [ ]:
BCH_P_4 = sum_collect(BCH_P_3,l)
BCH_P_4

-\omega⋅sin(\omega⋅t)⋅\hat{x} + cos(\omega⋅t)⋅\hat{p}

In [ ]:
BCH_sigma = BCH(H_2,sigmaz,alpha,7).subs({alpha:I*t})
BCH_sigma

     7                                               6                                             ↪
  ⅈ⋅t ⋅(-1024⋅\hat{\sigma}₊ + 1024⋅\hat{\sigma}₋)   t ⋅(-256⋅\hat{\sigma}₊ - 256⋅\hat{\sigma}₋ + 2 ↪
- ─────────────────────────────────────────────── - ────────────────────────────────────────────── ↪
                       5040                                                       720              ↪

↪                         5                                             4                          ↪
↪ 56⋅\hat{\sigma}_z)   ⅈ⋅t ⋅(-128⋅\hat{\sigma}₊ + 128⋅\hat{\sigma}₋)   t ⋅(-32⋅\hat{\sigma}₊ - 32⋅ ↪
↪ ────────────────── + ───────────────────────────────────────────── + ─────────────────────────── ↪
↪                                           120                                                    ↪

↪                                         3                                           2            ↪
↪ \hat{\sigma}₋ + 32⋅\hat{\sigma}_z)   ⅈ⋅t ⋅(-16⋅\hat{\sigma}₊ + 16⋅\hat{\sigma}₋)   t ⋅(

In [ ]:
BCH_sigma_1 = collect_all(BCH_sigma,l_2)
BCH_sigma_1

⎛      6      4           ⎞                  ⎛        7       6         5      4        3          ↪
⎜  16⋅t    4⋅t       2    ⎟                  ⎜  64⋅ⅈ⋅t    16⋅t    16⋅ⅈ⋅t    4⋅t    8⋅ⅈ⋅t       2   ↪
⎜- ───── + ──── - 2⋅t  + 1⎟⋅\hat{\sigma}_z + ⎜- ─────── + ───── + ─────── - ──── - ────── + 2⋅t  + ↪
⎝   45      3             ⎠                  ⎝    315      45       15       3       3             ↪

↪       ⎞                 ⎛      7       6         5      4        3               ⎞              
↪       ⎟                 ⎜64⋅ⅈ⋅t    16⋅t    16⋅ⅈ⋅t    4⋅t    8⋅ⅈ⋅t       2        ⎟              
↪  2⋅ⅈ⋅t⎟⋅\hat{\sigma}₋ + ⎜─────── + ───── - ─────── - ──── + ────── + 2⋅t  - 2⋅ⅈ⋅t⎟⋅\hat{\sigma}₊
↪       ⎠                 ⎝  315      45       15       3       3                  ⎠              

In [ ]:
(I*series(sin(t)**2,t,0,8).removeO()).expand()

     6      4       
2⋅ⅈ⋅t    ⅈ⋅t       2
────── - ──── + ⅈ⋅t 
  45      3         

In [ ]:
BCH_sigma_2 = collect_and_save_all(BCH_sigma,l_2)
BCH_sigma_2

⎡      6      4                   7       6         5      4        3                         7    ↪
⎢  16⋅t    4⋅t       2      64⋅ⅈ⋅t    16⋅t    16⋅ⅈ⋅t    4⋅t    8⋅ⅈ⋅t       2            64⋅ⅈ⋅t     ↪
⎢- ───── + ──── - 2⋅t  + 1, ─────── + ───── - ─────── - ──── + ────── + 2⋅t  - 2⋅ⅈ⋅t, - ─────── +  ↪
⎣   45      3                 315      45       15       3       3                        315      ↪

↪     6         5      4        3               ⎤
↪ 16⋅t    16⋅ⅈ⋅t    4⋅t    8⋅ⅈ⋅t       2        ⎥
↪ ───── + ─────── - ──── - ────── + 2⋅t  + 2⋅ⅈ⋅t⎥
↪  45       15       3       3                  ⎦

In [ ]:
BCH_sigma_3 = compare_all(BCH_sigma_2,l_2,7)
BCH_sigma_3

⎡      6      4                   7       6         5      4        3                         7    ↪
⎢  16⋅t    4⋅t       2      64⋅ⅈ⋅t    16⋅t    16⋅ⅈ⋅t    4⋅t    8⋅ⅈ⋅t       2            64⋅ⅈ⋅t     ↪
⎢- ───── + ──── - 2⋅t  + 1, ─────── + ───── - ─────── - ──── + ────── + 2⋅t  - 2⋅ⅈ⋅t, - ─────── +  ↪
⎣   45      3                 315      45       15       3       3                        315      ↪

↪     6         5      4        3               ⎤
↪ 16⋅t    16⋅ⅈ⋅t    4⋅t    8⋅ⅈ⋅t       2        ⎥
↪ ───── + ─────── - ──── - ────── + 2⋅t  + 2⋅ⅈ⋅t⎥
↪  45       15       3       3                  ⎦

In [ ]:
BCH_A = BCH(H_3,a_dag,alpha,7).subs({alpha:I*t})
BCH_A

          7  7                            6  6                              5  5                   ↪
  ⅈ⋅\omega ⋅t ⋅\hat{a}__{\dagger}   \omega ⋅t ⋅\hat{a}__{\dagger}   ⅈ⋅\omega ⋅t ⋅\hat{a}__{\dagger ↪
- ─────────────────────────────── - ───────────────────────────── + ────────────────────────────── ↪
               5040                              720                              120              ↪

↪           4  4                              3  3                            2  2                 ↪
↪ }   \omega ⋅t ⋅\hat{a}__{\dagger}   ⅈ⋅\omega ⋅t ⋅\hat{a}__{\dagger}   \omega ⋅t ⋅\hat{a}__{\dagg ↪
↪ ─ + ───────────────────────────── - ─────────────────────────────── - ────────────────────────── ↪
↪                  24                                6                                2            ↪

↪                                                         
↪ er}                                                     
↪ ─── + ⅈ⋅\omega⋅t⋅\hat{a}__{\dagger} + \hat{a}__{\dagger}
↪            

In [ ]:
BCH_A_1 = collect_all(BCH_A,l_3)
BCH_A_1

⎛          7  7         6  6           5  5         4  4           3  3         2  2               ↪
⎜  ⅈ⋅\omega ⋅t    \omega ⋅t    ⅈ⋅\omega ⋅t    \omega ⋅t    ⅈ⋅\omega ⋅t    \omega ⋅t                ↪
⎜- ──────────── - ────────── + ──────────── + ────────── - ──────────── - ────────── + ⅈ⋅\omega⋅t  ↪
⎝      5040          720           120            24            6             2                    ↪

↪    ⎞                   
↪    ⎟                   
↪ + 1⎟⋅\hat{a}__{\dagger}
↪    ⎠                   

In [ ]:
BCH_A_2 = collect_and_save_all(BCH_A,l_3)
BCH_A_2

⎡             7  7         6  6           5  5         4  4           3  3         2  2            ↪
⎢     ⅈ⋅\omega ⋅t    \omega ⋅t    ⅈ⋅\omega ⋅t    \omega ⋅t    ⅈ⋅\omega ⋅t    \omega ⋅t             ↪
⎢0, - ──────────── - ────────── + ──────────── + ────────── - ──────────── - ────────── + ⅈ⋅\omega ↪
⎣         5040          720           120            24            6             2                 ↪

↪       ⎤
↪       ⎥
↪ ⋅t + 1⎥
↪       ⎦

In [ ]:
BCH_A_3 = compare_all(BCH_A_2,l_3,7)
BCH_A_3

⎡             7  7         6  6           5  5         4  4           3  3         2  2            ↪
⎢     ⅈ⋅\omega ⋅t    \omega ⋅t    ⅈ⋅\omega ⋅t    \omega ⋅t    ⅈ⋅\omega ⋅t    \omega ⋅t             ↪
⎢0, - ──────────── - ────────── + ──────────── + ────────── - ──────────── - ────────── + ⅈ⋅\omega ↪
⎣         5040          720           120            24            6             2                 ↪

↪       ⎤
↪       ⎥
↪ ⋅t + 1⎥
↪       ⎦

In [ ]:
Latex(series(exp(-I*omega*t),t,0,8).removeO())

\frac{i \omega^{7} t^{7}}{5040} - \frac{\omega^{6} t^{6}}{720} - \frac{i \omega^{5} t^{5}}{120} + \frac{\omega^{4} t^{4}}{24} + \frac{i \omega^{3} t^{3}}{6} - \frac{\omega^{2} t^{2}}{2} - i \omega t + 1



$
\frac{i \omega^{7} t^{7}}{5040} - \frac{\omega^{6} t^{6}}{720} - \frac{i \omega^{5} t^{5}}{120} + \frac{\omega^{4} t^{4}}{24} + \frac{i \omega^{3} t^{3}}{6} - \frac{\omega^{2} t^{2}}{2} - i \omega t + 1
$


In [ ]:
def BCHxp(A,B,theta):
  bch = B+theta*conmutadorxp(A,B)+theta**2/factorial(2)*conmutadorxp(A,conmutadorxp(A,B))+theta**3/factorial(3)*conmutadorxp(A,conmutadorxp(A,conmutadorxp(A,B)))+theta**4/factorial(4)*conmutadorxp(A,conmutadorxp(A,conmutadorxp(A,conmutadorxp(A,B))))+theta**5/factorial(5)*conmutadorxp(A,conmutadorxp(A,conmutadorxp(A,conmutadorxp(A,conmutadorxp(A,B)))))+theta**6/factorial(6)*conmutadorxp(A,conmutadorxp(A,conmutadorxp(A,conmutadorxp(A,conmutadorxp(A,conmutadorxp(A,B))))))+theta**7/factorial(7)*conmutadorxp(A,conmutadorxp(A,conmutadorxp(A,conmutadorxp(A,conmutadorxp(A,conmutadorxp(A,conmutadorxp(A,B)))))))
  bchxa = collect_coeff(bch,x_1)
  bchxb = collect_coeff(bch,x_2)
  bchpa = collect_coeff(bch,p_1)
  bchpb = collect_coeff(bch,p_2)
  if bchxa.coeff(x_1)==series(cos(theta.args[1]),theta.args[1],0,8).removeO():
    bchxa = cos(theta.args[1])*x_1
  if bchxa.coeff(x_1)==series(sin(theta.args[1]),theta.args[1],0,8).removeO():
    bchxa = sin(theta.args[1])*x_1
  #---------------
  if bchxb.coeff(x_2)==series(cos(theta.args[1]),theta.args[1],0,8).removeO():
    bchxb = cos(theta.args[1])*x_2
  if bchxb.coeff(x_2)==series(sin(theta.args[1]),theta.args[1],0,8).removeO():
    bchxb = sin(theta.args[1])*x_2
  if bchxb.coeff(x_2)==series(-sin(theta.args[1]),theta.args[1],0,8).removeO():
    bchxb = -sin(theta.args[1])*x_2
  #---------------
  if bchpa.coeff(p_1)==series(cos(theta.args[1]),theta.args[1],0,8).removeO():
     bchpa = cos(theta.args[1])*p_1
  if bchpa.coeff(p_1)==series(sin(theta.args[1]),theta.args[1],0,8).removeO():
     bchpa = sin(theta.args[1])*p_1
  if bchpb.coeff(p_2)==series(cos(theta.args[1]),theta.args[1],0,8).removeO():
     bchpb = cos(theta.args[1])*p_2
  if bchpb.coeff(p_2)==series(sin(theta.args[1]),theta.args[1],0,8).removeO():
     bchpb = sin(theta.args[1])*p_2
  if bchpb.coeff(p_2)==series(-sin(theta.args[1]),theta.args[1],0,8).removeO():
     bchpb = -sin(theta.args[1])*p_2
  #-------------------------------
  if bchxa.coeff(x_1)==series(exp(2*theta.args[1]),theta.args[1],0,8).removeO():
     bchxa = exp(2*theta.args[1])*x_1
  if bchxb.coeff(x_2)==series(exp(2*theta.args[1]),theta.args[1],0,8).removeO():
     bchxb = exp(2*theta.args[1])*x_2
  if bchpa.coeff(p_1)==series(exp(-2*theta.args[1]),theta.args[1],0,8).removeO():
     bchpa = exp(-2*theta.args[1])*p_1
  if bchpb.coeff(p_2)==series(exp(-2*theta.args[1]),theta.args[1],0,8).removeO():
     bchpb = exp(-2*theta.args[1])*p_2
  return bchxa+bchxb+bchpa+bchpb

In [ ]:
def BHCexp(A,B,alpha,n):
  bhc = BCH(A, B, alpha, n)
  expression = collect_all(bhc,l)
  for i in range(len(l)):


